In [1]:
import os, sys, math, time, random
from pathlib import Path
import numpy as np
import pandas as pd
import torch
os.environ["CUDA_VISIBLE_DEVICES"] = "6" 
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, ConcatDataset, Subset, random_split
import torchvision.transforms as T
import torchvision.datasets as datasets
import timm
from tqdm import tqdm

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)
SEED = 42
if DEVICE == "cuda":
    print("GPU name:", torch.cuda.get_device_name(0))
    print("Total GPU mem (GB):", torch.cuda.get_device_properties(0).total_memory / (1024**3))

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if DEVICE == "cuda":
    torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

print("timm version:", timm.__version__)
print("PyTorch:", torch.__version__)

import os
from pathlib import Path
import torch
from ultralytics import YOLO
import json
import pprint

print("PyTorch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA device:", torch.cuda.get_device_name(0))

/home/23ucc611/miniconda3/envs/ml/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device: cuda
GPU name: Tesla V100-SXM2-32GB
Total GPU mem (GB): 31.7325439453125
timm version: 1.0.21
PyTorch: 2.6.0+cu124
PyTorch: 2.6.0+cu124
CUDA available: True
CUDA device: Tesla V100-SXM2-32GB


In [3]:
# Cell 1 — imports and paths (edit DATA_YAML if your yaml is elsewhere)
import os, sys, shutil, glob, yaml, re
from pathlib import Path
from collections import Counter

DATA_YAML = "/home/23ucc611/SWE/Poaching/dataset/data.yaml"   # <--- change if needed
assert os.path.exists(DATA_YAML), f"data.yaml not found at {DATA_YAML}"
BASE_DIR = str(Path(DATA_YAML).resolve().parent)   # dataset root (yaml parent)
print("Dataset root (BASE_DIR):", BASE_DIR)


Dataset root (BASE_DIR): /home/23ucc611/SWE/Poaching/dataset


In [4]:
# Cell 2 — parse data.yaml to get splits & class names
with open(DATA_YAML, 'r') as f:
    data = yaml.safe_load(f)
data


{'train': 'train/images',
 'val': 'valid/images',
 'test': 'test/images',
 'names': {0: 'Antelope',
  1: 'Badger',
  2: 'Bat',
  3: 'Bear',
  4: 'Bike',
  5: 'Binocular',
  6: 'Bison',
  7: 'Boar',
  8: 'Car',
  9: 'Cheetah',
  10: 'Chimpanzee',
  11: 'Coyote',
  12: 'Deer',
  13: 'Dog',
  14: 'Donkey',
  15: 'Duck',
  16: 'Eagle',
  17: 'Elephant',
  18: 'Flamingo',
  19: 'Fox',
  20: 'Giraffe',
  21: 'Goat',
  22: 'Goose',
  23: 'Gorilla',
  24: 'Hare',
  25: 'Hedgehog',
  26: 'Helicopter',
  27: 'Hippopotamus',
  28: 'Hornbill',
  29: 'Horse',
  30: 'Humming Bird',
  31: 'Hunter',
  32: 'Hyena',
  33: 'Jeep',
  34: 'Kangaroo',
  35: 'Knife',
  36: 'Koala',
  37: 'Leopard',
  38: 'Lion',
  39: 'Lizard',
  40: 'Mouse',
  41: 'Okapi',
  42: 'Orangutan',
  43: 'Otter',
  44: 'Owl',
  45: 'Ox',
  46: 'Panda',
  47: 'Parrot',
  48: 'Pig',
  49: 'Pigeon',
  50: 'Pistol',
  51: 'Porcupine',
  52: 'Possum',
  53: 'Raccoon',
  54: 'Reindeer',
  55: 'Rifle',
  56: 'Rinoceros',
  57: 'Rope',
  

In [6]:
# Cell 3 — infer split image/label directories (works for common layouts)
# tries to detect train/val/test or train/valid/test subfolders from yaml paths
def resolve_split_paths(field):
    p = data.get(field)
    if p is None: 
        return None
    p = str(Path(p))
    if not os.path.isabs(p):
        p = os.path.join(BASE_DIR, p)
    # common Roboflow layout: train/images, train/labels
    parent = Path(p).parent
    images = p if 'images' in p else os.path.join(parent, 'images')
    labels = os.path.join(parent, 'labels')
    return str(Path(images)), str(Path(labels))

train_images_dir, train_labels_dir = resolve_split_paths('train')
val_images_dir, val_labels_dir = resolve_split_paths('val') or resolve_split_paths('valid')
test_images_dir, test_labels_dir = resolve_split_paths('test') or (None, None)

print("train images:", train_images_dir)
print("train labels:", train_labels_dir)
print("val images  :", val_images_dir)
print("val labels  :", val_labels_dir)
print("test images :", test_images_dir)
print("test labels :", test_labels_dir)

CLASS_NAMES = data.get('names') or data.get('names', {})
# data may store names as list or dict
names_list = CLASS_NAMES
print("Num classes listed in data.yaml:", len(names_list) if names_list else "unknown")


train images: /home/23ucc611/SWE/Poaching/dataset/train/images
train labels: /home/23ucc611/SWE/Poaching/dataset/train/labels
val images  : /home/23ucc611/SWE/Poaching/dataset/valid/images
val labels  : /home/23ucc611/SWE/Poaching/dataset/valid/labels
test images : /home/23ucc611/SWE/Poaching/dataset/test/images
test labels : /home/23ucc611/SWE/Poaching/dataset/test/labels
Num classes listed in data.yaml: 72


In [7]:
# Cell 4 — helper: find images and labels and normalize names
# Supported image extensions:
IMG_EXTS = {'.jpg','.jpeg','.png','.bmp','.tif','.tiff','.webp','.gif'}
LABEL_EXTS = {'.txt'}   # YOLO label files

def list_files(dir_path, exts):
    if dir_path is None: return []
    d = Path(dir_path)
    if not d.exists():
        return []
    return [str(p) for p in d.rglob('*') if p.suffix.lower() in exts]

def normalize_filename(path):
    # replace spaces and problematic chars
    p = Path(path)
    new_name = re.sub(r'[\s]+', '_', p.name)  # replace spaces
    new_name = re.sub(r'[^0-9A-Za-z_.\-]', '', new_name)  # remove weird chars
    return str(p.with_name(new_name))

# collect
train_imgs = list_files(train_images_dir, IMG_EXTS)
train_lbls = list_files(train_labels_dir, LABEL_EXTS)
val_imgs   = list_files(val_images_dir, IMG_EXTS)
val_lbls   = list_files(val_labels_dir, LABEL_EXTS)
test_imgs  = list_files(test_images_dir, IMG_EXTS) if test_images_dir else []
test_lbls  = list_files(test_labels_dir, LABEL_EXTS) if test_labels_dir else []

print("counts -> train imgs:", len(train_imgs), "train lbls:", len(train_lbls))
print("counts -> val   imgs:", len(val_imgs), "val   lbls:", len(val_lbls))
print("counts -> test  imgs:", len(test_imgs), "test  lbls:", len(test_lbls))


counts -> train imgs: 4182 train lbls: 4182
counts -> val   imgs: 905 val   lbls: 905
counts -> test  imgs: 898 test  lbls: 898


In [8]:
# Cell 5 — rename files in-place to normalized names (images and labels)
# WARNING: this modifies filenames. Make a backup if you need to preserve originals.
def batch_rename(file_list):
    renamed = []
    for p in file_list:
        newp = normalize_filename(p)
        if newp != p:
            # ensure we don't overwrite existing file
            if os.path.exists(newp):
                # add suffix if collision
                base = str(Path(newp).with_suffix(''))
                sfx = 1
                while os.path.exists(f"{base}_{sfx}{Path(newp).suffix}"):
                    sfx += 1
                newp = f"{base}_{sfx}{Path(newp).suffix}"
            shutil.move(p, newp)
            renamed.append((p, newp))
    return renamed

renamed = []
renamed += batch_rename(train_imgs)
renamed += batch_rename(train_lbls)
renamed += batch_rename(val_imgs)
renamed += batch_rename(val_lbls)
renamed += batch_rename(test_imgs)
renamed += batch_rename(test_lbls)
print("Renamed files count:", len(renamed))
if renamed:
    print("Sample renames:", renamed[:10])
# refresh lists after rename
train_imgs = list_files(train_images_dir, IMG_EXTS)
train_lbls = list_files(train_labels_dir, LABEL_EXTS)
val_imgs = list_files(val_images_dir, IMG_EXTS)
val_lbls = list_files(val_labels_dir, LABEL_EXTS)
test_imgs = list_files(test_images_dir, IMG_EXTS)
test_lbls = list_files(test_labels_dir, LABEL_EXTS)


Renamed files count: 0


In [ ]:
# Cell 6 — match images <-> labels by base name, fix common mismatches (uppercase extensions, .jpeg vs .jpg), and report
def base(filepath):
    return Path(filepath).stem

def index_by_basename(file_list):
    d = {}
    for p in file_list:
        d[base(p)] = p
    return d

def find_matches(img_list, lbl_list):
    imgs_idx = index_by_basename(img_list)
    lbls_idx = index_by_basename(lbl_list)
    img_only = sorted([imgs_idx[k] for k in imgs_idx.keys() - lbls_idx.keys()])
    lbl_only = sorted([lbls_idx[k] for k in lbls_idx.keys() - imgs_idx.keys()])
    both = sorted([imgs_idx[k] for k in imgs_idx.keys() & lbls_idx.keys()])
    return img_only, lbl_only, both

img_only, lbl_only, both = (
    *find_matches(train_imgs, train_lbls),
)
print("TRAIN -> only-images (no label):", len(img_only))
print("TRAIN -> only-labels (no image):", len(lbl_only))
print("TRAIN -> matched pairs:", len(both))
if len(img_only)>0:
    print("sample img-only:", img_only[:5])
if len(lbl_only)>0:
    print("sample lbl-only:", lbl_only[:5])


TRAIN -> only-images (no label): 0
TRAIN -> only-labels (no image): 0
TRAIN -> matched pairs: 905


In [11]:
# Cell 8 — print top-level sanity and optionally create missing blank label files for images with no labels
# (create blank label if you want the model to treat it as 'no object' rather than fail)
CREATE_EMPTY_LABELS = False  # set True to auto-create .txt files for images missing labels (caution)
if CREATE_EMPTY_LABELS and img_only:
    for im_path in img_only:
        lbl_path = os.path.join(train_labels_dir, Path(im_path).with_suffix('.txt').name)
        if not os.path.exists(lbl_path):
            open(lbl_path,'w').close()
    print("Created empty label files for", len(img_only), "images.")
else:
    print("Empty label creation skipped. (set CREATE_EMPTY_LABELS=True to enable)")

# quick fail if too many mismatches
if len(img_only) > 0 or len(lbl_only) > 0:
    print("WARNING: There are unmatched images/labels in TRAIN. You may want to resolve before training.")
else:
    print("TRAIN image/label counts match exactly.")


Empty label creation skipped. (set CREATE_EMPTY_LABELS=True to enable)
TRAIN image/label counts match exactly.


In [12]:
# Cell 9 — Basic label format check: peek at some label files and detect OBB format
# OBB YOLOv8 common formats: 
# - center + w + h + angle: "cls cx cy w h angle" (5 floats after cls)
# - 8-point polygon: "cls x1 y1 x2 y2 x3 y3 x4 y4" (8 floats)
# We'll detect which it is.
def detect_label_format(sample_label_path):
    with open(sample_label_path) as f:
        lines = [l.strip() for l in f if l.strip()]
    if not lines:
        return "empty"
    parts = lines[0].split()
    floats_count = len(parts)-1
    if floats_count == 5:
        return "obb_cx_cy_w_h_angle"
    elif floats_count == 8:
        return "obb_8_points"
    elif floats_count == 4:
        return "yolo_xywh"  # standard axis-aligned
    else:
        return f"unknown ({floats_count} coords)"

sample_label = None
for p in train_lbls[:5]:
    if os.path.getsize(p) > 0:
        sample_label = p
        break

if sample_label:
    fmt = detect_label_format(sample_label)
    print("Detected label format for sample:", sample_label)
    print("Format:", fmt)
else:
    print("No non-empty label file found to detect format.")


Detected label format for sample: /home/23ucc611/SWE/Poaching/dataset/train/labels/0000015c-a1bb-dffb-a5de-adfb5a640003_4x3_jpg.rf.484d2188a05371551b31bdeeafc3e350.txt
Format: obb_8_points


In [13]:
# Cell 10 — per-class counts (approx; for OBB we read class id from start of each line)
def count_classes(label_paths):
    cnt = Counter()
    total = 0
    for p in label_paths:
        with open(p) as f:
            for l in f:
                l = l.strip()
                if not l: continue
                parts = l.split()
                cls = parts[0]
                cnt[int(float(cls))] += 1
                total += 1
    return total, cnt

train_total_objs, train_class_counts = count_classes(train_lbls)
val_total_objs,   val_class_counts   = count_classes(val_lbls)
print("TRAIN objects:", train_total_objs, "distinct classes in train:", len(train_class_counts))
print("VAL objects  :", val_total_objs,   "distinct classes in val:", len(val_class_counts))

# map index -> name if names available
if names_list:
    # convert names_list to list if dict earlier
    names = names_list
    print("\nTop train classes (id:name:count)")
    for cls_id, c in train_class_counts.most_common(15):
        name = names[cls_id] if cls_id < len(names) else "<unknown>"
        print(f"{cls_id}: {name}: {c}")
else:
    print("No class names found in data.yaml; showing ids only:")
    print(train_class_counts.most_common(20))


TRAIN objects: 5020 distinct classes in train: 72
VAL objects  : 1079 distinct classes in val: 72

Top train classes (id:name:count)
30: Humming Bird: 70
34: Kangaroo: 70
39: Lizard: 70
44: Owl: 70
32: Hyena: 70
36: Koala: 70
11: Coyote: 70
27: Hippopotamus: 70
9: Cheetah: 70
8: Car: 70
64: Truck: 70
33: Jeep: 70
68: Wombat: 70
43: Otter: 70
10: Chimpanzee: 70


In [14]:
# Cell 11 — create train.txt / val.txt / test.txt with absolute image paths (YOLO likes absolute or relative to data.yaml)
OUT_SPLIT_DIR = os.path.join(BASE_DIR, "splits_txt")
os.makedirs(OUT_SPLIT_DIR, exist_ok=True)

def write_split_list(img_list, out_path):
    # prefer absolute paths
    with open(out_path, 'w') as f:
        for p in sorted(img_list):
            f.write(str(Path(p).resolve()) + "\n")
    print("Wrote", out_path, " ->", len(img_list))

write_split_list(train_imgs, os.path.join(OUT_SPLIT_DIR, "train.txt"))
write_split_list(val_imgs, os.path.join(OUT_SPLIT_DIR, "val.txt"))
if test_imgs:
    write_split_list(test_imgs, os.path.join(OUT_SPLIT_DIR, "test.txt"))
print("Split files created at:", OUT_SPLIT_DIR)


Wrote /home/23ucc611/SWE/Poaching/dataset/splits_txt/train.txt  -> 4182
Wrote /home/23ucc611/SWE/Poaching/dataset/splits_txt/val.txt  -> 905
Wrote /home/23ucc611/SWE/Poaching/dataset/splits_txt/test.txt  -> 898
Split files created at: /home/23ucc611/SWE/Poaching/dataset/splits_txt


In [15]:
# Cell 12 — optional: create a backup copy of data.yaml that uses absolute paths for train/val/test (safer for CLI)
import copy
data_abs = copy.deepcopy(data)

def abs_path_entry(entry):
    if entry is None: return None
    p = str(Path(entry))
    if not os.path.isabs(p):
        p = str(Path(BASE_DIR) / p)
    return str(Path(p).resolve())

if data.get('train'):
    data_abs['train'] = abs_path_entry(data['train'])
if data.get('val'):
    data_abs['val'] = abs_path_entry(data.get('val') or data.get('valid') or data.get('val'))
if data.get('test'):
    data_abs['test'] = abs_path_entry(data['test'])

ABS_DATA_YAML = os.path.join(OUT_SPLIT_DIR, "data_abs.yaml")
with open(ABS_DATA_YAML,'w') as f:
    yaml.safe_dump(data_abs, f, sort_keys=False)
print("Wrote absolute-path yaml:", ABS_DATA_YAML)


Wrote absolute-path yaml: /home/23ucc611/SWE/Poaching/dataset/splits_txt/data_abs.yaml


In [18]:
# Cell 14 — show ready-to-run CLI command (edit epochs/imgsz/batch as needed)
# use ABS_DATA_YAML created earlier
model_name = "yolo11s-obb.pt"   # small OBB pretrained model (change to yolo11s-obb.pt if you want larger)
epochs = 50
imgsz = 1024   # OBB benefits from higher resolution on oriented objects; reduce if memory limited
batch = 16      # lower if OOM
project = "poacher_obb"
exp_name = "exp1"

cli_cmd = f"yolo obb train model={model_name} data={ABS_DATA_YAML} epochs={epochs} imgsz={imgsz} batch={batch} project={project} name={exp_name}"
print("Run this command in terminal (or prefix with '!' in a notebook cell):\n")
print(cli_cmd)
print("\nNotes:\n- If you run in Colab, prefix the command with '!' and ensure GPU is enabled.\n- If you get OOM, reduce imgsz or batch size.\n- Trained weights will be saved under runs/obb/train/{exp_name}/weights/")


Run this command in terminal (or prefix with '!' in a notebook cell):

yolo obb train model=yolo11s-obb.pt data=/home/23ucc611/SWE/Poaching/dataset/splits_txt/data_abs.yaml epochs=50 imgsz=1024 batch=16 project=poacher_obb name=exp1

Notes:
- If you run in Colab, prefix the command with '!' and ensure GPU is enabled.
- If you get OOM, reduce imgsz or batch size.
- Trained weights will be saved under runs/obb/train/{exp_name}/weights/


In [ ]:
# Cell 15 — optional: run training via the Python API (ultralytics) directly from notebook
# WARNING: this will start training inside the notebook kernel. Use this if you prefer an in-notebook run.
from ultralytics import YOLO

print("Preparing to start training via Python API. This call will block until training finishes (run only if you want to train in this notebook).")
print("Model:", model_name)
print("Data yaml:", ABS_DATA_YAML)
print("epochs, imgsz, batch:", epochs, imgsz, batch)

# Example: uncomment to run



Preparing to start training via Python API. This call will block until training finishes (run only if you want to train in this notebook).
Model: yolo11s-obb.pt
Data yaml: /home/23ucc611/SWE/Poaching/dataset/splits_txt/data_abs.yaml
epochs, imgsz, batch: 50 1024 16


In [20]:
model = YOLO(model_name)
model.train(data=ABS_DATA_YAML, epochs=epochs, imgsz=imgsz, batch=batch, project=project, name=exp_name)

New https://pypi.org/project/ultralytics/8.3.231 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.229 🚀 Python-3.10.18 torch-2.6.0+cu124 CUDA:0 (Tesla V100-SXM2-32GB, 32494MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/23ucc611/SWE/Poaching/dataset/splits_txt/data_abs.yaml, degrees=0.0, deterministic=True, device=6, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s-obb.pt, momentum=0.937, mosaic=1.0, multi_scale=Fa

ultralytics.utils.metrics.OBBMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f0ea3d97610>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027